# Speed Limits and Accidents in Minnesota
### Team Members:
- Robert
- Jon
- Calvin
- Mike

## Questions and Summary

**How do speed limits on Minnesota roads relate to accidents?**

1. Does the speed of a road lead to a higher number of accidents?

2. Is there a relationship between accident severity and speed limit?

    - As speed limit increases, frequency of accedents increases
        - Hypothesis testing shows statistical signifigance
        - Regression shows strong correlation
    - Severity data was not what we thought it was
        - Severity in dataset indicated impact on traffic
        - Analysis of severity did not lead to any meaningful conclusion


## Data Cleanup & Exploration

text here

## Data Analysis

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import time
import os
from scipy.stats import linregress
from citipy import citipy
import scipy.stats as stats

In [ ]:
# import csv
file = 'Resources/AccidentDataWithSpeed.csv'
df = pd.read_csv(file)
df.head()

In [ ]:
# Calvin's heat map starts here

In [ ]:
# Calvin's heat map ends here

In [ ]:
# Jon's graphs start here

In [ ]:
# Jon's graphs end here

In [ ]:
# Mike's hypothesis starts here

In [ ]:
speed_group = df.groupby("Speed Limit")
speed_count = speed_group["ID"].count().reset_index()
speed_count

In [ ]:
# place data into binned categories for hypothesis test
bins = [20, 35, 50, 70]
bin_labels = ["20-35", "35-50", ">50"]

speed_count["Bin"] = pd.cut(speed_count["Speed Limit"], bins, labels=bin_labels, include_lowest=True)
speed_count

In [ ]:
# calculate average number of accidents in each speed limit bin
speed_bin_group = speed_count.groupby("Bin")
speed_bin_group.mean()

In [ ]:
# place binned data into group variables for ANOVA test
group1 = speed_count[speed_count["Bin"] == "20-35"]["ID"]
group2 = speed_count[speed_count["Bin"] == "35-50"]["ID"]
group3 = speed_count[speed_count["Bin"] == ">50"]["ID"]


# perform ANOVA test
p_value = stats.f_oneway(group1, group2, group3)
print(f"p-value = {p_value[1]}")

In [ ]:
# create box plot of data
speed_count.boxplot("ID", by="Bin", figsize=(15,10))
plt.title("Accident Box Plot")
plt.xlabel("Speed Limits")
plt.ylabel("Count of Accidents")
plt.show()

In [ ]:
# Mike's hypothesis ends here

In [ ]:
# Roberts's regression starts here

In [ ]:
# Robert's regression ends here

## Discussion

## Post Mortem

## Questions